# Quantitative Value Strategy

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [8]:
import numpy as np, pandas as pd, requests, xlsxwriter, math
from scipy import stats

In [54]:
# Importing our list of stocks
stocks = pd.read_csv("data/sp_500_stocks.csv")
# filter out delisted stocks
stocks = stocks[~stocks['Ticker'].isin(["AAL", "AAP", "AABV",'DISCA', 'HFC','VIAC','WLTW', 'CTL', "ETFC", "MYL", "NBL", "A"])]

stocks['Ticker']
from secrets_new import IEX_CLOUD_API_TOKEN 

In [55]:
# Making our first API call
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 90659836,
 'calculationPrice': 'close',
 'change': 10.97,
 'changePercent': 0.07756,
 'close': 0,
 'closeSource': 'iocflfai',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 156.52,
 'iexCloseTime': 1737176807600,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 155.87,
 'iexOpenTime': 1734433387501,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1677026722581,
 'latestPrice': 160.27,
 'latestSource': 'Close',
 'latestTime': 'October 28, 2022',
 'latestUpdate': 1720802742400,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 2570292973127,
 'oddLotD

In [56]:
# Parsing our API call
pe_ratio = data['peRatio']
pe_ratio

25.93

In [57]:
# Exectuing a batch API call
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\2106919777.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield lst[i:i + n]


In [58]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\L

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\L

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\L

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\L

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\octav\AppData\L

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AAPL,155.99,25.97,N/A
1,ABBV,149.70,21.89,N/A
2,ABC,160.27,18.93,N/A
3,ABMD,270.54,56.16,N/A
4,ABT,103.63,21.06,N/A
...,...,...,...,...
489,YUM,122.50,23.93,N/A
490,ZBH,113.63,105.69,N/A
491,ZBRA,288.00,31.39,N/A
492,ZION,52.61,6.06,N/A


# Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.


In [42]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [43]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,45.91,3.18,N/A
1,AIG,59.6,3.75,N/A
2,PHM,40.75,4.1,N/A
3,PVH,53.23,4.1,N/A
4,NUE,139,4.32,N/A
5,COF,110.1,4.7,N/A
6,DISH,16.1,4.76,N/A
7,LYB,79.79,4.78,N/A
8,HPQ,28.27,4.86,N/A
9,AIV,8.1,4.91,N/A


In [23]:
# calculaitng the number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")


In [24]:
portfolio_input()

Enter the value of your portfolio:1000000


In [27]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])


final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,PSX,104.60,8.87,191
1,UNM,46.17,8.65,433
2,EMN,79.39,8.53,251
3,MPC,114.50,8.48,174
4,VZ,38.25,8.31,522
5,KEY,18.60,8,1075
6,GS,344.96,7.83,57
7,WDC,35.50,7.77,563
8,WBA,37.82,7.5,528
9,MRO,30.47,7.4,656


## Building a Better (and More Realistic) Value Strategy

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
    Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.


In [44]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [45]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

In [60]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_35776\3686472774.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
C:\Users\octav\AppData\Local\Temp\ipykernel_

In [61]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,160.77,N/A,25.84,N/A,43.15,N/A,6.7,N/A,19.963117,N/A,15.646469,N/A,N/A
1,AAPL,159.36,N/A,25.84,N/A,43.67,N/A,6.74,N/A,20.438817,N/A,15.311568,N/A,N/A
2,ABBV,149.27,N/A,21.46,N/A,18.0,N/A,4.72,N/A,11.789868,N/A,8.291306,N/A,N/A
3,ABC,161.83,N/A,19.18,N/A,151.05,N/A,0.1434,N/A,10.114240,N/A,4.237957,N/A,N/A
4,ABMD,264.06,N/A,55.7,N/A,8.0,N/A,11.19,N/A,37.991028,N/A,12.490157,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,120.87,N/A,23.83,N/A,-4.05,N/A,5.13,N/A,20.025857,N/A,9.027220,N/A,N/A
491,ZBH,119.10,N/A,106.72,N/A,2,N/A,3.33,N/A,17.858378,N/A,5.604420,N/A,N/A
492,ZBRA,299.00,N/A,30.79,N/A,5.95,N/A,2.66,N/A,15.388003,N/A,6.601598,N/A,N/A
493,ZION,53.18,N/A,5.93,N/A,1.56,N/A,2.31,N/A,4.601827,N/A,2.211576,N/A,N/A


In [63]:
# Dealing with missing values in our dataframe
# Let's forst identify the misssing data
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
26,ALXN,184.80,N/A,59.63,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
38,AON,292.20,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
69,BRK.B,309.43,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
86,CERN,98.63,N/A,49.07,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTXS,106.10,N/A,41.16,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
132,DISCK,24.71,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
181,FLIR,57.52,N/A,33.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FOX,27.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
188,FRT,102.32,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
199,GOOG,96.79,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


There are two main approaches:

    Drop missing data from the data set (pandas' dropna method is useful here)
    Replace missing data with a new value (pandas' fillna method is useful here)

In this tutorial, we will replace missing data with the average non-NaN data point from that column.

Here is the code to do this:

In [64]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)


In [65]:
#' Check to see if we still have any missing data
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

    Price-to-earnings ratio
    Price-to-book ratio
    Price-to-sales ratio
    EV/EBITDA
    EV/GP

Here's how we'll do this:

In [67]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe


0      0.631313
1      0.631313
2      0.511111
3      0.458586
4      0.921212
         ...   
490    0.565657
491    0.977778
492    0.735354
493    0.084848
494     0.80404
Name: PE Percentile, Length: 495, dtype: object
0      0.977778
1      0.979798
2      0.933333
3      0.993939
4      0.793939
         ...   
490    0.058586
491    0.319192
492    0.709091
493    0.214141
494    0.890909
Name: PB Percentile, Length: 495, dtype: object
0      0.858586
1      0.860606
2      0.763636
3       0.00404
4      0.957576
         ...   
490     0.80202
491    0.594949
492    0.476768
493    0.426263
494    0.925253
Name: PS Percentile, Length: 495, dtype: object
0      0.785859
1      0.806061
2      0.393939
3      0.315152
4      0.967677
         ...   
490    0.789899
491    0.725253
492    0.628283
493    0.082828
494     0.90101
Name: EV/EBITDA Percentile, Length: 495, dtype: object
0      0.929293
1      0.923232
2      0.656566
3      0.309091
4      0.866667
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,160.77,N/A,25.84,0.631313,43.15,0.977778,6.7000,0.858586,19.963117,0.785859,15.646469,0.929293,N/A
1,AAPL,159.36,N/A,25.84,0.631313,43.67,0.979798,6.7400,0.860606,20.438817,0.806061,15.311568,0.923232,N/A
2,ABBV,149.27,N/A,21.46,0.511111,18.00,0.933333,4.7200,0.763636,11.789868,0.393939,8.291306,0.656566,N/A
3,ABC,161.83,N/A,19.18,0.458586,151.05,0.993939,0.1434,0.00404,10.114240,0.315152,4.237957,0.309091,N/A
4,ABMD,264.06,N/A,55.70,0.921212,8.00,0.793939,11.1900,0.957576,37.991028,0.967677,12.490157,0.866667,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,120.87,N/A,23.83,0.565657,-4.05,0.058586,5.1300,0.80202,20.025857,0.789899,9.027220,0.70101,N/A
491,ZBH,119.10,N/A,106.72,0.977778,2.00,0.319192,3.3300,0.594949,17.858378,0.725253,5.604420,0.422222,N/A
492,ZBRA,299.00,N/A,30.79,0.735354,5.95,0.709091,2.6600,0.476768,15.388003,0.628283,6.601598,0.49899,N/A
493,ZION,53.18,N/A,5.93,0.084848,1.56,0.214141,2.3100,0.426263,4.601827,0.082828,2.211576,0.129293,N/A


## Calculating the RV Score

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.


In [98]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAPL,160.77,N/A,25.84,0.631313,43.15,0.977778,6.7000,0.858586,19.963117,0.785859,15.646469,0.929293,0.836566
1,AAPL,159.36,N/A,25.84,0.631313,43.67,0.979798,6.7400,0.860606,20.438817,0.806061,15.311568,0.923232,0.840202
2,ABBV,149.27,N/A,21.46,0.511111,18.00,0.933333,4.7200,0.763636,11.789868,0.393939,8.291306,0.656566,0.651717
3,ABC,161.83,N/A,19.18,0.458586,151.05,0.993939,0.1434,0.00404,10.114240,0.315152,4.237957,0.309091,0.416162
4,ABMD,264.06,N/A,55.70,0.921212,8.00,0.793939,11.1900,0.957576,37.991028,0.967677,12.490157,0.866667,0.901414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,YUM,120.87,N/A,23.83,0.565657,-4.05,0.058586,5.1300,0.80202,20.025857,0.789899,9.027220,0.70101,0.583434
491,ZBH,119.10,N/A,106.72,0.977778,2.00,0.319192,3.3300,0.594949,17.858378,0.725253,5.604420,0.422222,0.607879
492,ZBRA,299.00,N/A,30.79,0.735354,5.95,0.709091,2.6600,0.476768,15.388003,0.628283,6.601598,0.49899,0.609697
493,ZION,53.18,N/A,5.93,0.084848,1.56,0.214141,2.3100,0.426263,4.601827,0.082828,2.211576,0.129293,0.187475


In [99]:
# Selecting the best 50 stocks
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [100]:
# Create a dummy portfoio to get the number of shares to buy
portfolio_input()

position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

Enter the value of your portfolio:1000000


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,54.29,368,4.00,0.050505,0.6629,0.072727,0.3881,0.042424,4.371626,0.074747,0.920363,0.036364,0.055354
1,KSS,31.36,637,6.03,0.088889,0.8089,0.080808,0.1968,0.006061,4.188411,0.062626,1.073688,0.056566,0.05899
2,AIG,57.10,350,3.80,0.048485,0.9925,0.10101,0.7557,0.115152,2.166570,0.018182,0.752996,0.024242,0.061414
3,GM,39.46,506,6.78,0.107071,0.8556,0.084848,0.3918,0.044444,2.025873,0.016162,1.621050,0.094949,0.069495
4,BA,149.08,134,-10.12,0.032323,-4.8500,0.056566,1.4000,0.258586,-24.143538,0.00404,-153.357874,0.00202,0.070707
5,F,13.48,1483,6.21,0.094949,1.2700,0.145455,0.3586,0.032323,3.964926,0.050505,1.835627,0.10101,0.084848
6,C,47.23,423,6.10,0.090909,0.4978,0.066667,1.0800,0.174747,3.601498,0.038384,1.118711,0.058586,0.085859
7,COF,109.80,182,4.91,0.060606,0.7755,0.076768,1.2500,0.222222,3.138818,0.028283,1.232907,0.068687,0.091313
8,UAA,7.46,2680,9.61,0.165657,1.4300,0.189899,0.5237,0.066667,2.772690,0.020202,0.628331,0.014141,0.091313
9,PHM,41.89,477,4.30,0.054545,1.1800,0.128283,0.6190,0.086869,3.490204,0.036364,2.482947,0.161616,0.093535


In [101]:
# Formatting our Excel output
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [102]:


background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )



In [103]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [104]:
writer.save()

C:\Users\octav\AppData\Local\Temp\ipykernel_35776\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
